애니메이션 데이터 세트에서 클러스터링 기술을 사용하는 추천 시스템.
클러스터의 특성에서 추천 애니메이션을 추출.
사용자는 사용자의 애니메이션 평가 기록에 따라 분류(사용자가 좋아하는 애니만 선택)

* Data wrangling
* K mean clustering
* Characteristic of each cluster

**anime.csv**

* anime_id: 애니메이션을 식별하는 myanimelist.net의 고유 ID입니다.
* name: 애니메이션의 전체 이름
* genre: 이 애니메이션의 쉼표로 구분 된 장르 목록입니다.
* type: 영화, TV, OVA 등
* episodes: 해당 애니메니션의 에피소드 수. (영화인 경우 1).
* rating: 해당 애니메이션에 대한 평가는 10 점 만점.
* members: 해당 애니메이션의 "그룹"에있는 커뮤니티 회원 수.

**Rating.csv**

* user_id: 식별 할 수없는 무작위로 생성 된 사용자 ID.
* anime_id: 이 사용자가 평가 한 애니메이션.
* rating: 이 사용자가 할당한 10점 내 평가 (사용자가 봤지만 평가를 지정하지 않은 경우 -1).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline

plt.rcParams['figure.figsize'] = (6,4)
plt.style.use('ggplot')
%config InlineBackend.figure_formats = {'png', 'retina'}

In [ ]:
anime = pd.read_csv('../input/anime-recommendations-database/anime.csv')

In [ ]:
anime.shape

In [ ]:
anime.describe()

In [ ]:
anime.head(20)

In [ ]:
user = pd.read_csv('../input/anime-recommendations-database/rating.csv')

In [ ]:
user.head(10)

In [ ]:
user.shape

In [ ]:
user['user_id'].unique()

In [ ]:
user.describe()

<h3>Like의 정의</h3>

사용자가 많아 애니메이션 평가에 대한 많은 차이가 있음.<br>
각 사용자의 평가 평균을 찾기로 결정.<br>
사용자 평점 평균보다 높은 등급을받은 애니메이션은 다음과 같이 지정.

In [ ]:
user[user['user_id']==1].rating.mean()

In [ ]:
user[user['user_id']==2].rating.mean()

In [ ]:
user[user['user_id']==5].rating.mean()

사용자당 평균 평가계산

In [ ]:
MRPU = user.groupby(['user_id']).mean().reset_index()
MRPU['mean_rating'] = MRPU['rating']

MRPU.drop(['anime_id', 'rating'], axis=1, inplace=True)

In [ ]:
MRPU.head(10)

In [ ]:
MRPU.shape

In [ ]:
user = pd.merge(user,MRPU,on=['user_id','user_id'])

In [ ]:
user.head()

In [ ]:
user = user.drop(user[user.rating < user.mean_rating].index)

In [ ]:
user.head()

In [ ]:
user.shape

In [ ]:
user = user.rename({'rating':'userRating'}, axis='columns')

**두 개의 데이터 세트 결합**

이 커널에서는 실행 시간으로 인해 데이터 세트의 크기를 줄입니다

In [ ]:
mergedata = pd.merge(anime,user,on=['anime_id','anime_id'])
mergedata = mergedata[mergedata.user_id <= 20000]
mergedata.head(10)

In [ ]:
len(mergedata['anime_id'].unique())

In [ ]:
len(anime['anime_id'].unique())

크로스테이블 생성

각 사용자가 좋아하는 애니메이션 세부 정보 표시

In [ ]:
user_anime = pd.crosstab(mergedata['user_id'], mergedata['name'])
user_anime.head(10)

In [ ]:
user_anime.shape

주요 구성 요소 분석

기존 변수를 새로운 변수 세트로 변환.<br>
기존 변수 세트의 선형 조합입니다.<br>
이 커널의 주요 목표는 클러스터링을위한 데이터 차원을 줄이고 시각화


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(user_anime)
pca_samples = pca.transform(user_anime)

In [ ]:
ps = pd.DataFrame(pca_samples)
ps.head()

In [ ]:
tocluster = pd.DataFrame(ps[[0,1,2]])

In [ ]:
plt.rcParams['figure.figsize'] = (16,9)

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(tocluster[0], tocluster[2], tocluster[1])


plt.title('Data points in 3D PCA axis', fontsize=20)
plt.show()

<h3>K 평균</h3>

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

scores = []
inertia_list = np.empty(8)

for i in range(2,8):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(tocluster)
    inertia_list[i] = kmeans.inertia_
    scores.append(silhouette_score(tocluster, kmeans.labels_))
 

In [ ]:
plt.plot(range(0,8),inertia_list,'-o')
plt.xlabel('Number of cluster')
plt.axvline(x=4, color='blue', linestyle='--')
plt.ylabel('Inertia')
plt.show()


In [ ]:
plt.plot(range(2,8), scores);
plt.title('Results KMeans')
plt.xlabel('n_clusters');
plt.axvline(x=4, color='blue', linestyle='--')
plt.ylabel('Silhouette Score');
plt.show()


<h3>K means clustering</h3>

In [ ]:
from sklearn.cluster import KMeans

clusterer = KMeans(n_clusters=4,random_state=30).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)

print(centers)


In [ ]:
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(tocluster[0], tocluster[2], tocluster[1], c = c_preds)
plt.title('Data points in 3D PCA axis', fontsize=20)

plt.show()


In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(tocluster[1],tocluster[0],c = c_preds)
for ci,c in enumerate(centers):
    plt.plot(c[1], c[0], 'o', markersize=8, color='red', alpha=1)

plt.xlabel('x_values')
plt.ylabel('y_values')

plt.title('Data points in 2D PCA axis', fontsize=20)
plt.show()


In [ ]:
user_anime['cluster'] = c_preds


user_anime.head(10)

<h3>각 클러스터의 특징</h3>

In [ ]:
c0 = user_anime[user_anime['cluster']==0].drop('cluster',axis=1).mean()
c1 = user_anime[user_anime['cluster']==1].drop('cluster',axis=1).mean()
c2 = user_anime[user_anime['cluster']==2].drop('cluster',axis=1).mean()
c3 = user_anime[user_anime['cluster']==3].drop('cluster',axis=1).mean()


<h3>Cluster 0</h3>
이 클러스터의 특징을 설명하는 톱 15 애니메이션


In [ ]:
c0.sort_values(ascending=False)[0:15]


In [ ]:
def createAnimeInfoList(animelist):
    episode_list = list()
    genre_list = list()
    member_list = list()
    rating_list= list()
    for x in anime['name']:
        if x in animelist:
            episode_list.append(anime[anime['name']==x].episodes.values.astype(int))
            member_list.append(anime[anime['name']==x].members.values.astype(int))
            rating_list.append(anime[anime['name']==x].rating.values.astype(int))
            for y in anime[anime['name']==x].genre.values:
                 genre_list.append(y)
    return genre_list,episode_list,rating_list,member_list
 

In [ ]:
def count_word(df, ref_col, liste):
    keyword_count = dict()
    for s in liste: keyword_count[s] = 0
    for liste_keywords in df[ref_col].str.split(','):        
        if type(liste_keywords) == float and pd.isnull(liste_keywords): continue        
        for s in [s for s in liste_keywords if s in liste]: 
            if pd.notnull(s): keyword_count[s] += 1
    #______________________________________________________________________
    # convert the dictionary in a list to sort the keywords by frequency
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count


In [ ]:
animelist = list(c0.index)
data = pd.DataFrame()
data['genre'],data['episode'],data['rating'],data['member'] =  createAnimeInfoList(animelist)


In [ ]:
set_keywords = set()
for liste_keywords in data['genre'].str.split(',').values:
    if isinstance(liste_keywords, float): continue  # only happen if liste_keywords = NaN
    set_keywords = set_keywords.union(liste_keywords)


In [ ]:
from wordcloud import WordCloud

def makeCloud(Dict,name,color):
    words = dict()

    for s in Dict:
        words[s[0]] = s[1]

        wordcloud = WordCloud(
                      width=1500,
                      height=500, 
                      background_color=color, 
                      max_words=20,
                      max_font_size=500, 
                      normalize_plurals=False)
        wordcloud.generate_from_frequencies(words)


    fig = plt.figure(figsize=(12, 8))
    plt.title(name)
    plt.imshow(wordcloud)
    plt.axis('off')

    plt.show()


In [ ]:
c0_animelist = list(c0.sort_values(ascending=False)[0:15].index)
c0_data = pd.DataFrame()
c0_data['genre'],c0_data['episode'],c0_data['rating'],c0_data['member'] =  createAnimeInfoList(c0_animelist)
c0_data.iloc[:,1:4] = c0_data.iloc[:,1:4].astype(int) # change to numeric object to integer
keyword_occurences, dum = count_word(c0_data, 'genre', set_keywords)
makeCloud(keyword_occurences[0:10],"cluster 0","lemonchiffon")


In [ ]:
keyword_occurences[0:5]


In [ ]:
print('cluster 0\nAVG episode : {0}\nAVG movie rating : {1}\nAVG member : {2}'
      .format(c0_data['episode'].mean(), c0_data['rating'].mean(),c0_data['member'].mean()))


<h3>Cluster 1</h3>

In [ ]:
c1.sort_values(ascending=False)[0:15]


In [ ]:
c1_animelist = list(c1.sort_values(ascending=False)[0:15].index)
c1_data = pd.DataFrame()
c1_data['genre'],c1_data['episode'],c1_data['rating'],c1_data['member'] =  createAnimeInfoList(c1_animelist)
c1_data.iloc[:,1:4] = c1_data.iloc[:,1:4].astype(int)
keyword_occurences, dum = count_word(c1_data, 'genre', set_keywords)
makeCloud(keyword_occurences[0:10],"cluster 1","white")


In [ ]:
keyword_occurences[0:5]


In [ ]:
print('cluster 1\nAVG episode : {0}\nAVG movie rating : {1}\nAVG member : {2}'
      .format(c1_data['episode'].mean(), c1_data['rating'].mean(),c1_data['member'].mean()))


<h3>cluster 2</h3>

In [ ]:
c2.sort_values(ascending=False)[0:15]


In [ ]:
c2_animelist = list(c2.sort_values(ascending=False)[0:15].index)
c2_data = pd.DataFrame()
c2_data['genre'],c2_data['episode'],c2_data['rating'],c2_data['member'] =  createAnimeInfoList(c2_animelist)
c2_data.iloc[:,1:4] = c2_data.iloc[:,1:4].astype(int)
keyword_occurences, dum = count_word(c2_data, 'genre', set_keywords)
makeCloud(keyword_occurences[0:10],"cluster 2","black")


In [ ]:
keyword_occurences[0:5]


In [ ]:
c2_data['episode'].mean()


In [ ]:
print('cluster 2\nAVG episode : {0}\nAVG movie rating : {1}\nAVG member : {2}'
      .format(c2_data['episode'].mean(), c2_data['rating'].mean(),c2_data['member'].mean()))


<h3>Cluster 3</h3>

In [ ]:
c3.sort_values(ascending=False)[0:15]


In [ ]:
c3_animelist = list(c3.sort_values(ascending=False)[0:15].index)
c3_data = pd.DataFrame()
c3_data['genre'],c3_data['episode'],c3_data['rating'],c3_data['member'] =  createAnimeInfoList(c3_animelist)
c3_data.iloc[:,1:4] = c3_data.iloc[:,1:4].astype(int)
keyword_occurences, dum = count_word(c3_data, 'genre', set_keywords)
makeCloud(keyword_occurences[0:10],"cluster 3","snow")


In [ ]:
keyword_occurences[0:5]


In [ ]:
print('cluster 3\nAVG episode : {0}\nAVG movie rating : {1}\nAVG member : {2}'
      .format(c3_data['episode'].mean(), c3_data['rating'].mean(),c3_data['member'].mean()))
